In [1]:
#same imports
import math
import numpy as np
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [2]:
#unchanged
def euclid(p1, p2):
    return math.hypot(p1.x - p2.x, p1.y - p2.y)

In [3]:
# finding the gaze ratios  to get the values suited for the 
# index
def gazeratios(landmarks):
    iris=landmarks[474]
    inner=landmarks[263]
    outer=landmarks[362]
    eyewidth=euclid(inner, outer)
    if eyewidth==0:
        return 0.5, 0.5  #center 
    horizontal = euclid(iris, inner) / eye_width
    xs = np.array([lm.x for lm in landmarks])# i am converting this x position into array based positions for better computation
    ys = np.array([lm.y for lm in landmarks])
    # choose candidate landmarks with x close to the iris.x
    candidates = [i for i in range(len(landmarks)) if abs(landmarks[i].x - iris.x) < x_tol]
    if not candidates:
        top = min(inner.y, outer.y)
        bottom = max(inner.y, outer.y)
    else:
        candidate_ys = [landmarks[i].y for i in candidates]
        top = min(candidate_ys)
        bottom = max(candidate_ys)
    # avoid division by zero
    denom = (bottom - top) if (bottom - top) > 1e-6 else 1e-6 # extreme condition
    vertical = (iris.y - top_y) / denom
    # return normalized ratios
    return horizontal, vertical
    
# same what i did in the last notebook
def browratio(landmarks):
    brow_l = landmarks[107]
    brow_r = landmarks[336]
    face_l = landmarks[234]
    face_r = landmarks[454]
    facewidth = euclid(face_l, face_r) if euclid(face_l, face_r) > 1e-6 else 1e-6
    return euclid(brow_l, brow_r) / facewidth

def mouthmetrics(landmarks):
    # mouth width normalized, and vertical mouth openness
    left = landmarks[61]; right = landmarks[291]
    mouthwidth = euclid(left, right)
    facewidth = euclid(landmarks[234], landmarks[454]) if euclid(landmarks[234], landmarks[454])>1e-6 else 1e-6
    mouth_ratio = mouth_width / face_width
    # smile proxy: are corners higher (smile) relative to nose base?
    nose = landmarks[1]
    corner_avg_y = (left.y + right.y) / 2.0
    smile_upness = (nose.y - corner_avg_y) / face_width  # larger -> corners higher (smile)
    # lip opening
    top = landmarks[13]; bottom = landmarks[14]
    mouth_open = euclid(top, bottom) / face_width
    return mouth_ratio, smile_upness, mouth_open
# the head roll angle
def headrollangle(landmarks):
    L = landmarks[234]; R = landmarks[454]
    dx = R.x - L.x
    dy = R.y - L.y
    # roll angle in degrees: positive when right cheek lower than left i would try better by showing in the demo
    angle = math.degrees(math.atan2(dy, dx))
    return angle